In [ ]:
import time
import requests
import json

def fetch_schedule():
    schedule_url = "https://statsapi.mlb.com/api/v1/schedule?sportId=1&season=2024"
    response = requests.get(schedule_url)
    return response.json()

def detect_updates(current_data, new_data):
    if current_data != new_data:
        print("New data detected!")
        # Process the updated data here
        return new_data
    return current_data

# Polling loop
def start_polling(interval=60):  # Check every 60 seconds
    print("Starting polling...")
    current_data = fetch_schedule()

    while True:
        time.sleep(interval)
        try:
            new_data = fetch_schedule()
            new_data['new_data'] = "New Data"
            current_data = detect_updates(current_data, new_data)
        except Exception as e:
            print(f"Error fetching data: {e}")

start_polling()
